In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pydrake.all import (
    DiagramBuilder,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    Propeller,
    PropellerInfo,
    RigidTransform,
    RobotDiagramBuilder,
    SceneGraph,
    Simulator,
    Meshcat,
    StartMeshcat,
    namedview,
)
from pydrake.examples import QuadrotorGeometry, QuadrotorPlant, StabilizingLQRController
import numpy as np

running_as_notebook = True
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [2]:
from pydrake.all import DiagramBuilder
from pydrake.examples import QuadrotorGeometry as QuadrotorGeometry_EA
from control.UtilLeafSystems import NonConstantVectorSource

from control.QuadrotorControllers import QuadrotorLQR_EA
from sim.Quadrotor import MakeMultibodyQuadrotor_EA
from maths.quaternions import SampleQuaternion
from IPython.display import SVG, display, clear_output, Markdown
from time import time 


def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time()  # Start time before the function call
        result = func(*args, **kwargs)  # Call the function
        end_time = time()  # End time after the function call
        elapsed_time = end_time - start_time  # Calculate the time difference
        print(f"Function '{func.__name__}' executed in {elapsed_time:.4f} seconds")
        return result  # Return the original function's result
    return wrapper

Q = np.diag(np.concatenate(([10] * 6, [1] * 6)))
R = np.eye(4)

builder = DiagramBuilder()

# Init Systems
quadrotor, mbp = MakeMultibodyQuadrotor_EA(show_diagram=False)
quadrotor_system = builder.AddSystem(quadrotor)
controller = builder.AddSystem(QuadrotorLQR_EA(quadrotor_system, mbp, Q, R))
goal_state_source = builder.AddSystem(NonConstantVectorSource(12))
ref_action_source = builder.AddSystem(NonConstantVectorSource(4))


# Controller input connections
builder.Connect(
    quadrotor_system.get_output_port(), controller.get_input_port(0)
)
builder.Connect(
    goal_state_source.get_output_port(), controller.get_input_port(1)
)
builder.Connect(
    ref_action_source.get_output_port(), controller.get_input_port(2)
)

# Controller output connections
builder.Connect(
    controller.get_output_port(), quadrotor_system.get_input_port()
)

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
quadrotor_geometry_system = QuadrotorGeometry_EA.AddToBuilder(
    builder, quadrotor_system.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
# display(SVG(pydot.graph_from_dot_data(
#             diagram.GetGraphvizString(max_depth=1))[0].create_svg()))

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
diagram_context = simulator.get_mutable_context()

# Get the subsystem contexts
quadrotor_context = quadrotor_system.GetMyContextFromRoot(diagram_context)
controller_context = controller.GetMyContextFromRoot(diagram_context)
geometry_context = diagram.GetSubsystemContext(quadrotor_geometry_system, diagram_context)

# Function to print the state vector
def dynamically_update_output(state, control):
    clear_output(wait=True)
    display(Markdown(f"**State:**\n```\n{state}\n```\n**Control:**\n```\n{control}\n```"))

def log_state():
    quadrotor_state_port = quadrotor_system.get_output_port(0)
    control_output_port = controller.get_output_port(0)

    quadrotor_state = quadrotor_state_port.Eval(quadrotor_context)
    control = control_output_port.Eval(controller_context)

    return quadrotor_state, control

def simulate(dt, diagram_context, initial_state, ref_state, ref_action, duration):
    diagram_context.SetTime(0.0)
    diagram_context.SetContinuousState(initial_state)
    goal_state_source.SetState(ref_state)
    ref_action_source.SetState(ref_action) 
    simulator.Initialize()
    try:
        state_log = initial_state
        control_log = None
        
        while diagram_context.get_time() < duration:
            state, control = log_state()
            state_log = np.vstack((state_log, state))
            control_log = np.vstack((control_log, control)) if control_log is not None else control
            simulator.AdvanceTo(diagram_context.get_time() + dt)
    except ValueError as e:
        print(f"Simulation error: {e}")

    return state_log, control_log

# initial_state = np.array([1., 0., 0., 0., 0., 0., 1.5, 0., 0., 0., 0., 0., 0.])
initial_state = np.hstack(
    (np.array([0., 0., 0.8]),
     .5* np.random.randn(3),
    np.array([0., 0., 0., 0., 0., 0.])
    )
)

ref_state = np.array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
gravity = mbp.gravity_field().gravity_vector()[2]
ref_action = np.array([-controller.mass * gravity / 4 for i in range(4)] )

0.775


In [3]:


#################
@timeit
def Rollout(x0: np.ndarray, U: np.ndarray, dt: float, timeit: bool = False):
    """
    Simulates the trajectory of states (rollout) for given initial state and control sequence.

    :param x0: initial state, ndarray of shape (nx,)
    :param U: sequence of actions, ndarray of shape (num_time_steps-1, nu)
    :param dt: 

    :return x: sequence of states from rollout, ndarray of shape (num_time_steps, nx)
    """

    X = [x0.copy().tolist()]


    simulator = Simulator(quadrotor)
    simulator.Initialize()
    simulator_context = simulator.get_mutable_context()
    simulator_context.SetContinuousState(x0)

    for u in U:
        # Set input and simulate a small time step

        quadrotor.get_input_port().FixValue(simulator_context, u)
        
        # Use the plant's dynamics (or a numerical integrator) to step forward in time
        # This should ideally be a single step of the plant's integrator
        # simulator = Simulator(self.plant, context)
        sim_time = simulator_context.get_time()

        simulator.AdvanceTo(sim_time + dt)
        x_next = simulator_context.get_continuous_state_vector().CopyToVector()

        X.append(x_next)

        # print(f"time: {sim_time + dt} \nx_next: {x_next} \nu: {u} \n")

    return np.array(X)
#########

# Quadrotor params
# Kinda lazy to hardcode but what can you do ¯\_(ツ)_/¯
L = 0.15  # Length of the arms (m).
kF = 1.0  # Force input constant.
kM = 0.0245  # Moment input constant.
m = 0.775
MoI = np.array([[0.0015, 0, 0,],
                        [0, 0.0025, 0],  
                        [0, 0, 0.0035]])
inv_MoI = np.linalg.inv(MoI)

def ContinuousDynamics(x, u):
        '''
        :param x: state as [x, y, z, yaw, pitch, roll, x_dot, y_dot, z_dot, yaw_rate, pitch_rate, roll_rate]^T
        :param u: action as motor currents
        :return x_dot: state time derivative
        '''
        #State
        euler_angles = x[3:6]
        linear_velocity = x[6:9]
        angular_velocity = x[9:]
        rpy_angular_velocity = np.flip(angular_velocity)
        y,  p,  r = euler_angles.tolist()
        cr, cp, cy = np.cos(r), np.cos(p), np.cos(y)
        sr, sp, sy = np.sin(r), np.sin(p), np.sin(y)

        # Rotation matrix from body frame to world frame
        R_NB = np.array([[      cp*cy,        cy*sp*sr - sy*cr,    cy*sp*cr + sy*sr],
                        [       sy*cp,        sy*sp*sr + cy*cr,    sy*sp*cr - cy*sr],
                        [        -sp,               cp*sr,               cr*cp    ]])

        # =====================Linear Accel Calc===================== #

        # Compute thrust due to rotors in body frame
        uF_Bz = kF * u 
        Faero_B = np.array([0., 0., np.sum(uF_Bz)], dtype = x.dtype) 

        # Compute gravity and world frame
        Fgrav_N = np.array([0, 0, -m * 9.81], dtype = x.dtype)

        # Total forces
        Ftot_N = Fgrav_N + R_NB @ Faero_B

        linear_accel = Ftot_N / m

        # ====================Angular Accel Calc===================== #

        # Compute moments due to thrust 
        Mx_B = L * (uF_Bz[1] - uF_Bz[3])
        My_B = L * (uF_Bz[2] - uF_Bz[0])

        # Compute reaction moments due to actuating motors (Motor direction accounted for in summing)
        uTau_Bz = kM * u
        Mz_B = uTau_Bz[0] - uTau_Bz[1] + uTau_Bz[2] - uTau_Bz[3]

        # Total torque
        Tau_B = np.array([Mx_B, My_B, Mz_B], dtype = x.dtype)

        # Base angular velocity in world, expressed in base coordinates
        M = np.array([[1.,    0.,   -sp],
                        [0.,    cr,  sr*cp],
                        [0.,   -sr,  cr*cp]], dtype = x.dtype)

        w_BN_B = M @ rpy_angular_velocity

        # Compute angular acceleration in body frame, expressed in world coordinates
        alpha_NB_N = R_NB @ inv_MoI @ (Tau_B - np.cross(w_BN_B, MoI @ w_BN_B) )
        
        
        # Compute angular accleration in world coordinates
        Minv = np.array([[cy/cp,      sy/cp,     0.0],
                        [  -sy,         cy,      0.0],
                        [cy/cp*sp,   sy/cp*sp,   1.0]], dtype = x.dtype)
        
        yaw_rate, pitch_rate, roll_rate = angular_velocity

        Mdt =  np.array([[-cy*sp*pitch_rate - sy*cp*yaw_rate,   -cy*yaw_rate,    0.0,],
                         [-sy*sp*pitch_rate + cy*cp*yaw_rate,   -sy*yaw_rate,    0.0],
                         [           -cp*pitch_rate,                 0.0,        0.0]], dtype = x.dtype)
        
        angular_accel = np.flip(Minv @ (alpha_NB_N - Mdt @ rpy_angular_velocity))
        

        return np.hstack((linear_velocity, angular_velocity, linear_accel, angular_accel))     



from maths.forward_integration import RK4
def discrete_dynamics(x0, u, dt):
    xnext = ContinuousDynamics(x0, u)*dt + x0
    # xdot = ContinuousDynamics(x0, u)
    # xnext= RK4(ContinuousDynamics, x0, u, dt )
    roll, pitch, yaw = xnext[3:6]

    # if roll > np.pi:
    #     roll -= 2*np.pi
    # elif roll < -np.pi:
    #     roll += 2*np.pi 

    # if pitch > np.pi/2:
    #     pass
    # elif pitch < -np.pi/2:
    #     pass

    # if yaw > np.pi:
    #     yaw -= 2*np.pi
    # elif yaw < -np.pi:
    #     yaw += 2*np.pi
    return xnext

@timeit
def AnalyticRollout(x0: np.ndarray, U: np.ndarray, dt: float, timeit: bool = False):
    xtraj = x0.copy()
    x = x0.copy()
    for u in U:
        x = discrete_dynamics(x, u, dt)
        xtraj = np.vstack((xtraj, x))
    return xtraj 

from pydrake.examples import QuadrotorPlant


from pydrake.all import ContinuousState, BasicVector

cpp_quadrotor = QuadrotorPlant()
P = np.array(
    [
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
    ]
)
PP = np.block([[P, np.zeros((6, 6))], [np.zeros((6, 6)), P]])

def CompareDerivatives(x, u):

    # print(x)
    analyticDerivative = ContinuousDynamics(x, u)
    # context = quadrotor_system.CreateDefaultContext()
    # mbpDerivative = context.get_mutable_continuous_state()
    # context.SetContinuousState(x)
    # quadrotor.get_input_port().FixValue(context, u)
    # mbpDerivative = quadrotor.EvalTimeDerivatives(context)

    context = cpp_quadrotor.CreateDefaultContext()
    cpp_quadrotor.get_input_port().FixValue(context, u)
    context.SetContinuousState(PP@x)
    mbpDerivative = PP@cpp_quadrotor.EvalTimeDerivatives(context).CopyToVector()

    return np.linalg.norm(analyticDerivative - mbpDerivative), analyticDerivative, mbpDerivative

In [4]:
np.set_printoptions(linewidth=200)

# print(mbp.GetStateNames())
dt = 0.001
state_log, control_log = simulate(dt, diagram_context, initial_state, ref_state, ref_action, duration = 1.0)
x0 = state_log[0,:]
x_simulated = Rollout(x0, control_log, dt)
x_analytic = AnalyticRollout(x0, control_log, dt)

for x_sim, x_ana in zip(x_simulated, x_analytic):
    print(f"x_sim: {x_sim}")
    print(f"x_ana: {x_ana}")
    print()

# for i in range(len(x_analytic)-1):
#     err, analDer, mbpDer = CompareDerivatives(state_log[i,:], control_log[i,:])
#     # print(np.linalg.norm(analDer[9:]))
#     # print(np.linalg.norm(mbpDer[9:]))

#     perErr = np.abs(analDer-mbpDer)/mbpDer
#     # print(err)
#     # print(np.abs(analDer - mbpDer))
#     print(np.abs(analDer-mbpDer)/mbpDer)
#     # print(np.mean(perErr[-3:]))
#     # print(analDer)
#     # print(mbpDer)
#     print()

Function 'Rollout' executed in 0.0334 seconds
Function 'AnalyticRollout' executed in 0.0403 seconds
x_sim: [ 0.          0.          0.8        -0.36284099  0.8664701   0.39032863  0.          0.          0.          0.          0.          0.        ]
x_ana: [ 0.          0.          0.8        -0.36284099  0.8664701   0.39032863  0.          0.          0.          0.          0.          0.        ]

x_sim: [ 2.99664342e-06 -3.46591332e-06  7.99998521e-01 -3.62999404e-01  8.66199532e-01  3.89993780e-01  5.99318973e-03 -6.93135015e-03 -2.95721198e-03 -3.16647990e-01 -5.41162609e-01 -6.69528147e-01]
x_ana: [ 0.          0.          0.8        -0.36284099  0.8664701   0.39032863  0.00599337 -0.00693223 -0.00295825 -0.31699385 -0.54110118 -0.66984926]

x_sim: [ 1.19897412e-05 -1.38653487e-05  7.99994092e-01 -3.63456268e-01  8.65408426e-01  3.89047835e-01  1.19926793e-02 -1.38661700e-02 -5.89979179e-03 -5.96621099e-01 -1.04112637e+00 -1.22193442e+00]
x_ana: [ 5.99336817e-06 -6.93222589e-

Casadi example

In [5]:
import casadi as ca
import numpy as np

# Define symbolic variables
x = ca.MX.sym('x')  # state
u = ca.MX.sym('u')  # control input

# Define a simple nonlinear dynamical system
# For example, dx/dt = x + u^2 (a simple 1D system with quadratic control input)
dynamics = x + u**2

# Compute the Jacobian of the dynamics with respect to x
jacobian_x = ca.jacobian(dynamics, x)

# Compute the Jacobian of the dynamics with respect to u
jacobian_u = ca.jacobian(dynamics, u)

# Display the Jacobians
print("Jacobian with respect to x:")
print(jacobian_x)

print("\nJacobian with respect to u:")
print(jacobian_u)

# Now, let's evaluate these Jacobians for specific values of x and u
x_val = 1.0
u_val = 2.0

# Substitute the values of x and u into the Jacobians using ca.subs
jacobian_x_val = ca.substitute(jacobian_x, x, x_val)
jacobian_u_val = ca.substitute(jacobian_u, u, u_val)

# Evaluate the numerical values
jacobian_x_num = ca.evalf(jacobian_x_val)
jacobian_u_num = ca.evalf(jacobian_u_val)

# Display the evaluated Jacobians
print("\nEvaluated Jacobian with respect to x at x = {}, u = {}:".format(x_val, u_val))
print(jacobian_x_num)

print("\nEvaluated Jacobian with respect to u at x = {}, u = {}:".format(x_val, u_val))
print(jacobian_u_num)


Jacobian with respect to x:
1

Jacobian with respect to u:
(2.*u)

Evaluated Jacobian with respect to x at x = 1.0, u = 2.0:
1

Evaluated Jacobian with respect to u at x = 1.0, u = 2.0:
4


In [18]:
import casadi as ca
import numpy as np


# Quadrotor parameters 
L = 0.15  # Length of the arms (m).
kF = 1.0  # Force input constant.
kM = 0.0245  # Moment input constant.
m = 0.775
MoI = np.array([[0.0015, 0, 0],
                [0, 0.0025, 0],
                [0, 0, 0.0035]])
inv_MoI = np.linalg.inv(MoI)

# Define symbolic variables
x_sym = ca.MX.sym('state', 12)  # State [x, y, z, yaw, pitch, roll, x_dot, y_dot, z_dot, yaw_rate, pitch_rate, roll_rate]
u_sym = ca.MX.sym('action', 4)   # Control inputs (motor currents)

# Extract parts of the state vector
euler_angles = x_sym[3:6]
linear_velocity = x_sym[6:9]
angular_velocity = x_sym[9:]
rpy_angular_velocity = ca.vertcat(angular_velocity[2], angular_velocity[1], angular_velocity[0])

# Define rotation matrix R_NB from body to world frame
y, p, r = euler_angles[0], euler_angles[1], euler_angles[2]
cr, cp, cy = ca.cos(r), ca.cos(p), ca.cos(y)
sr, sp, sy = ca.sin(r), ca.sin(p), ca.sin(y)

R_NB = ca.vertcat(
    ca.horzcat(cp*cy, cy*sp*sr - sy*cr, cy*sp*cr + sy*sr),
    ca.horzcat(sy*cp, sy*sp*sr + cy*cr, sy*sp*cr - cy*sr),
    ca.horzcat(  -sp,       cp*sr,             cr*cp)
)
# Compute thrust due to rotors
uF_Bz = kF * u_sym
# Faero_B = ca.MX([0, 0, uF_Bz[0] + uF_Bz[1] + uF_Bz[2]])
Faero_B = ca.MX.zeros(1, 3)
Faero_B[0,-1] = uF_Bz[0] + uF_Bz[1] + uF_Bz[2] + uF_Bz[3]

# Compute gravity and world frame
Fgrav_N = ca.MX([0, 0, -m * 9.81])

# Total forces
Ftot_N = Fgrav_N + R_NB @ Faero_B.T
linear_accel = Ftot_N / m

# Compute moments
Mx_B = L * (uF_Bz[1] - uF_Bz[3])
My_B = L * (uF_Bz[2] - uF_Bz[0])
uTau_Bz = kM * u_sym
Mz_B = uTau_Bz[0] - uTau_Bz[1] + uTau_Bz[2] - uTau_Bz[3]
Tau_B = ca.horzcat(Mx_B, My_B, Mz_B)

# Base angular velocity in world
M = ca.vertcat(
    ca.horzcat(1., 0., -sp),
    ca.horzcat(0., cr, sr*cp),
    ca.horzcat(0., -sr, cr*cp)
)
w_BN_B = M @ rpy_angular_velocity

# Angular acceleration in body frame
alpha_NB_N = R_NB @ inv_MoI @ (Tau_B - ca.cross(w_BN_B, MoI @ w_BN_B).T).T

# Compute angular acceleration in world coordinates
Minv = ca.vertcat(
    ca.horzcat(cy/cp, sy/cp, 0.0),
    ca.horzcat(-sy, cy, 0.0),
    ca.horzcat(cy/cp*sp, sy/cp*sp, 1.0)
)

angular_accel = Minv @ alpha_NB_N

yaw_rate, pitch_rate = angular_velocity[0], angular_velocity[1]

Mdt = ca.vertcat(
    ca.horzcat(-cy*sp*pitch_rate - sy*cp*yaw_rate,   -cy*yaw_rate,    0.0),
    ca.horzcat(-sy*sp*pitch_rate + cy*cp*yaw_rate,   -sy*yaw_rate,    0.0),
    ca.horzcat( -cp*pitch_rate,                 0.0,        0.0)
)

rpy_angular_accel = Minv @ (alpha_NB_N - Mdt @ rpy_angular_velocity)
angular_accel = ca.vertcat(rpy_angular_accel[2], rpy_angular_accel[1], rpy_angular_accel[0])

# Define the system dynamics
x_dot = ca.vertcat(linear_velocity, angular_velocity, linear_accel, angular_accel)


# Compute Jacobians (A and B matrices)
A = ca.jacobian(x_dot, x_sym)  # Jacobian w.r.t. state (x)
B = ca.jacobian(x_dot, u_sym)  # Jacobian w.r.t. control input (u)


xdot_fun = ca.Function('A', [x_sym, u_sym], [x_dot])
# Now, you can create a CasADi function for A and B matrices
A_fun = ca.Function('A', [x_sym, u_sym], [A])
B_fun = ca.Function('B', [x_sym, u_sym], [B])

# Example state and control input for linearization
x0 = np.zeros(12)  # Example state (can be a trim condition)
u0 = np.ones(4)    # Example control input (nominal values)

# Evaluate the Jacobian at the example state and control input
A_matrix = A_fun(x0, u0)
B_matrix = B_fun(x0, u0)

# Convert to numpy arrays
A_matrix_np = np.array(A_matrix.full())
B_matrix_np = np.array(B_matrix.full())

print("A Matrix:\n", A_matrix_np)
print("B Matrix:\n", B_matrix_np)


action
A Matrix:
 [[ 0.          0.          0.          0.          0.          0.          1.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.          1.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.          0.          1.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.          0.          0.          1.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          1.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          1.        ]
 [ 0.          0.          0.          0.          5.16129032  0.          0.          0.          0

In [20]:
np.set_printoptions(linewidth=200)
# print(mbp.GetStateNames())
dt = 0.001
state_log, control_log = simulate(dt, diagram_context, initial_state, ref_state, ref_action, duration = 1.0)
# x0 = state_log[0,:]
# x_simulated = Rollout(x0, control_log, dt)
# x_analytic = AnalyticRollout(x0, control_log, dt)

# for x_sim, x_ana in zip(x_simulated, x_analytic):
#     print(f"x_sim: {x_sim}")
#     print(f"x_ana: {x_ana}")
#     print()


avg_anal = 0
avg_cas = 0
for i in range(len(x_analytic)-1):
    x, u = state_log[i,:], control_log[i,:]

    start = time()
    ContinuousDynamics(x, u)
    avg_anal += time()-start
    start = time()
    xdot_fun(x, u)
    avg_cas += time()-start
avg_anal /= i+1
avg_cas /= i+1

print(f"Avg Analytic: {avg_anal}s")
print(f"Avg CasADi: {avg_cas}s")

Avg Analytic: 4.2481899261474607e-05s
Avg CasADi: 1.7944574356079102e-05s


In [27]:
from pydrake.all import FirstOrderTaylorApproximation

@timeit
def drakeLin(ref_state, ref_action):
    context = quadrotor.CreateDefaultContext()
    context.SetContinuousState(ref_state)

    quadrotor.get_input_port().FixValue(context, ref_action)
    sys =   FirstOrderTaylorApproximation(quadrotor, 
                            context,
                            quadrotor.get_input_port().get_index(),
                            quadrotor.get_output_port().get_index())
    print(sys.A()) 
    print(sys.B())

# Evaluate the Jacobian at the example state and control input
@timeit
def casLin(ref_state, ref_action):
    A_matrix = A_fun(ref_state, ref_action)
    B_matrix = B_fun(ref_state, ref_action)
    print(A_matrix.full())
    print(B_matrix.full())

drakeLin(ref_state, ref_action)
casLin(ref_state, ref_action)

[[ 0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.  ]
 [-0.   -0.   -0.   -0.    9.81 -0.   -0.   -0.   -0.   -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -9.81 -0.   -0.   -0.   -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.  ]]
[[   0.            0.            0.            0.        ]
 [   0.            0.            0.     